# First RL Gym Project

## 1. Imports

In [1]:
import torch as tr
import torch.nn as nn
from torch.optim.lr_scheduler import MultiStepLR
import matplotlib.pyplot as plt
import numpy as np
from environments import BoxEnvironment1 as env
from environment_utils import Box
from agents import SACAgent
from agent_utils import update_target_agent, ReplayBuffer
from log_utils import RLLogger
from plot_utils import plot_run

device = tr.device('cuda' if tr.cuda.is_available() else 'cpu')
tr.autograd.set_detect_anomaly(True)
tr.set_default_tensor_type(tr.FloatTensor)

## 2. Hyperparameters

In [2]:
# ---------------- Training -------------------
    # Memory
memory_size = 64
memory_batch_size = 8 
    # Duration of training
runs = 1
n_episodes = 10
n_steps = 256 + memory_size
    # Training parameters
agent_batch_size = 1
learning_rate = 0.001
learning_rate_decay = 0.8
entropy_coef = 0.2
    # Bellman equation
future_discount = 0.8
    # Update Target Model
target_model_update = n_episodes//10
    # Loss Function
loss_function = nn.MSELoss()

# ---------------- Environment  ----------------
    # Environment box size
env_width = 2
env_height = 2
space = Box(env_width, env_height)
    # Goal box size and center
goal_width = 0.2
goal_height = 0.2
goal_center = np.tile([0.5,0],(agent_batch_size,1))
goal = Box(goal_width, goal_height, goal_center)
    # Time step size
dt = 0.0375
    # Noise
noise_characteristic_length = 1
    # Maximum of potential
U0 = 0.4

# ---------------- Agent ----------------------
state_dim = 5
hidden_dims = [16,16]
act_dim = 1
act_positive = True
act_scaling = 2*np.pi

# ---------------- Other ----------------------
plt.rcParams.update({'font.size': 13})
plt.rcParams.update({'figure.dpi': 150})
total_time = []
update_state_time = []

## 3. Simulation

In [3]:
environment = env(space, goal)
memory = ReplayBuffer(state_dim, act_dim, memory_size, agent_batch_size)
agent = SACAgent(state_dim, act_dim, hidden_dims, act_scaling, act_positive).float().to(device)
target_agent = SACAgent(state_dim, act_dim, hidden_dims, act_scaling, act_positive).float().to(device)
logger = RLLogger()

agent.actor_optimizer = tr.optim.Adam(agent.actor.parameters(), lr=learning_rate)
agent.critic1_optimizer = tr.optim.Adam(agent.critic1.parameters(), lr=learning_rate)
agent.critic2_optimizer = tr.optim.Adam(agent.critic2.parameters(), lr=learning_rate)

for p in target_agent.parameters():
    p.requires_grad = False

In [4]:
def update(agent, target_agent, memory_batch):
    state_now = memory_batch['state_now']
    state_next = memory_batch['state_next']
    action_now = memory_batch['action_now']
    reward = memory_batch['reward']
    done = memory_batch['done']
    action_next, log_prob_next = agent.actor(state_next)
    
    # Compute Prediction
    Q1_now = agent.critic1(state_now, action_now)
    Q2_now = agent.critic2(state_now, action_now)
    Q_now = tr.min(Q1_now, Q2_now)

    # Compute Target
    Q1_next = target_agent.critic1(state_next, action_next)
    Q2_next = target_agent.critic2(state_next, action_next)
    Q_next = tr.min(Q1_next, Q2_next)
    Target = reward + future_discount*(Q_next - entropy_coef*log_prob_next)

    if tr.isnan(Q1_next).any():
        print('Q1_next is nana')
    if tr.isnan(Q2_next).any():
        print('Q2_next is nana')
    if tr.isnan(Q_next).any():
        print('Q_next is nana')
    if tr.isnan(Target).any():
        print('Target is nana')
    if tr.isnan(log_prob_next).any():
        print('log_prob_next is nana')
    if tr.isnan(reward).any():
        print('reward is nana')
     
    # Compute Loss
    loss = loss_function(Q_now, Target)
    
    # Update
    loss.backward()
    agent.critic1_optimizer.step()
    agent.critic2_optimizer.step()
    agent.actor_optimizer.step()
    return loss

In [5]:
def episode():    
    environment.init_state(agent_batch_size)
    for current_step in range(n_steps):
        logger.save_step(environment.state)
        if current_step%target_model_update == 0 and current_step > memory_size:
            update_target_agent(agent, target_agent)
            # Beginning state
        state_now = environment.state
            # Action
        action_now, _ = agent.actor(tr.as_tensor(environment.state, device=device, dtype=tr.float))
            # Next state
        environment.step(action_now.detach().cpu().numpy(), U0, dt)
        state_next = environment.state
            # Reward
        reward = environment.reward(dt)
            # Done
        done = environment.goal_check()
            # Store in memory
        memory.store(state_now, action_now, reward, state_next, done)

            # Sample from memory
        if current_step > memory_size:
            memory_batch = memory.sample_batch(memory_batch_size)
                # Update Agent
            loss = update(agent, target_agent, memory_batch)
            logger.save_loss(loss)

        if max(environment.goal_check()): 
            print('Goal reached')
            print(f'Enviroment State: {environment.state}')
            logger.save_step(environment.state)
            logger.save_episode(current_step)
            break

In [6]:
def simulation():
    update_target_agent(agent, target_agent)
    for ep in range(n_episodes):
        episode()
        print('Episode', ep,' finished!')

simulation()

Episode 0  finished!
Episode 1  finished!
Episode 2  finished!
Episode 3  finished!
Episode 4  finished!
Episode 5  finished!
Episode 6  finished!
Episode 7  finished!
Episode 8  finished!
Episode 9  finished!


## Plotting


In [10]:
plt.plot(logger.losses.cpu().numpy())
plt.xlabel('step')
plt.ylabel('loss')
plt.title('Loss over time')
plt.show()

AttributeError: 'list' object has no attribute 'cpu'